In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from IPython.core.debugger import set_trace
%matplotlib notebook

In [ ]:
BASE_PATH = './pack/data/matches/'

In [ ]:
def find_nearest_idx(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [ ]:
class Game():
    def __init__(self, game_id):
        self.game_id = game_id
        self.players = None # Dictionary with keys of player ids
        self.events = None # List of events
        self.id_data = None
        self.id_kills = None
        self.constructGame(game_id)
        
    def constructGame(self, game_id):
        dataFile = BASE_PATH + game_id + '/ticks.csv'
        id_data = pd.read_csv(dataFile)
        self.id_data = id_data
                
        killFile = BASE_PATH + game_id + '/kills.csv'
        id_kills = pd.read_csv(killFile)
        self.id_kills = id_kills
        
        self.loadPlayers()
        self.loadEvents()
    
    def loadPlayers(self):
        killer_ids = np.unique(self.id_kills['killer_recording_id'])

        killers = {}
        for killer in killer_ids:
            kills = self.id_kills['killer_recording_id'] == killer
            killers[killer] = self.id_kills[kills]

        player_ids = np.unique(self.id_data['recording_id'])

        players = {}

        for player_id in player_ids:
            player = Player(player_id)

            # Save player's ticks
            tick_indices = self.id_data['recording_id'] == player_id
            player.tick_indices = tick_indices

            # gathering all the player's kills
            kill_indices = self.id_kills['killer_recording_id'] == player_id
            player.kill_indices = kill_indices

            # Save kill timestamps
            kill_stamps = self.player_kill_ticks(player.tick_indices, player.kill_indices)
            player.kill_stamps = kill_stamps

            players[player_id] = player

        self.players = players
    
    def loadEvents(self):

        def event_tick_index(player_id, event_stamp):
            ticks_indices = self.id_data['recording_id'] == player_id
            player_timestamps = self.id_data['timestamp'][ticks_indices]
            tick_index = find_nearest_idx(player_timestamps, event_stamp)
            return tick_index

        # Construct events
        events = {}
        for idx in range(0, len(self.id_kills)):
            row = self.id_kills.iloc[idx]
            event_id = idx
            killer_id = row['killer_recording_id']
            victim_id = row['victim_recording_id']
            kill_stamp = row['timestamp']
            killer_tick_index = event_tick_index(killer_id, kill_stamp)
            victim_tick_index = event_tick_index(victim_id, kill_stamp)
            event = Event(event_id, killer_tick_index, victim_tick_index, killer_id, victim_id)
            events[event_id] = event
        
        self.events = events
        
    def player_kill_ticks(self, tick_indices, kill_indices):

        kill_stamps = self.id_kills['timestamp'][kill_indices]
        timestamps = self.id_data['timestamp'][tick_indices]
        kill_ticks_stamps = []
        for i, kill_stamp in enumerate(kill_stamps):    
            kill_tick_idx = find_nearest_idx(timestamps, kill_stamp)
            kill_ticks_stamps.append(timestamps[kill_tick_idx])
        return kill_ticks_stamps
    
    def player_kill_events(self, player_id):
        events = {}
        for event_key in self.events:
            event = events[event_key]
            if (event[killer_id] == player_id):
                timestamp = event.getKillerTimestamp()
                victim_id = event.victim_id
                events.add([timestamp, victim_id])
        return events
    
    def player_death_events(self, player_id, ):
        events = {}
        for event_key in self.events:
            event = events[event_key]
            if (event[victim_id] == player_id):
                timestamp = event.getVictimTimestamp()
                killer_id = event.killer_id
                events.add([timestamp, killer_id])
        return events
        
    def player_log(self, player_id):
        player = self.players[player_id]
        moves = player.list_moves(self.id_data)
        return moves
    
    def build_game_log(self):
        log = {'players_log':{},
              'events_log': {}}
        for player_id in self.players:
            log['players_log'][player_id] = player_log(player_id)
            for event_log in player_kill_events(player_id):
                timestamp = event_log[0]
                victim = event_log[1]
                log['players_log'][player_id][timestamp]['kill'] = True
                log['players_log'][player_id][timestamp]['victim'] = victim

            for event_log in player_kill_events(player_id):
                timestamp = event_log[0]
                killer = event_log[1]
                log['players_log'][player_id][timestamp]['death'] = True
                log['players_log'][player_id][timestamp]['killer'] = victim

        return log

In [ ]:
class Player():
    def __init__(self, player_id):
        self.player_id = player_id
        self.tick_indices = None
        self.kill_indices = None
        self.kill_stamps = None
    
    def list_moves(self, id_data):
        player_ticks = id_data[self.tick_indices]
        moves = {}
        for step in player_ticks:
            timestep = step['timestep']
            moves[timestep] = {step}
        return moves

In [ ]:
class Event():
    # Kill event
    def __init__(self, event_id, killer_tick_index, victim_tick_index, killer_id, victim_id):
        self.event_id = event_id
        self.killer_id = killer_id
        self.victim_id = victim_id
        self.killer_tick_index = killer_tick_index # Index for tick for killer
        self.victim_tick_index = victim_tick_index
        
    def getKillerTimestamp(self, id_data):
        return id_data.iloc[self.killer_tick_index]['timestamp']
    
    def getVictimTimestamp(self, id_data):
        return id_data.iloc[self.victim_tick_index]['timestamp']
    
    def player_kill_pos(self, id_data):
        pos_x = id_data['pos_x'][self.killer_tick_index]
        pos_y = id_data['pos_y'][self.killer_tick_index]
        return float(pos_x), float(pos_y)
    
    def player_death_pos(self, id_data):
        pos_x = id_data['pos_x'][self.victim_tick_index]
        pos_y = id_data['pos_y'][self.victim_tick_index]
        return float(pos_x), float(pos_y)
        

In [ ]:
games = []

matches = os.listdir(BASE_PATH)
idx = 1
total_matches = len(matches)
# Assumes that all items in this directory are match folders
for match_id in matches:
    if match_id[0] == '.':
        # Skip .DS_Store file
        total_matches -= 1
        continue
    
    game = Game(match_id)
    games.append(game)
    
    print 'Loaded game ' + str(idx) + ' of ' + str(total_matches)
    idx += 1
    
    if idx > 5:
        break
    

In [ ]:
# plot kills

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
# ([0, 0, 1, 1])
bg_figname = 'foolsaas_v1.png'
img = plt.imread(bg_figname)
axis_extent = (-246400, 159990,
               -266400, 140000)

idx = 1
for game in games:
    plt.subplot(len(games), 2, idx)
    plt.imshow(img, extent=axis_extent)
    position_data_x = []
    position_data_y = []


    for event in game.events:
        event = game.events[event]
        pos_x, pos_y = event.player_kill_pos(game.id_data)
        plt.scatter(pos_x, pos_y, c='r', s=5)
        position_data_x.append(pos_x)
        position_data_y.append(pos_y)
        
    heatmap, xedges, yedges = np.histogram2d(position_data_x, position_data_y, bins=50)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.subplot(len(games), 2, idx+1)
    plt.imshow(heatmap.T, extent=extent, origin='lower')
    plt.show()


    
    idx += 2


In [ ]:
# plot movement
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
# ([0, 0, 1, 1])
bg_figname = 'foolsaas_v1.png'
img = plt.imread(bg_figname)
axis_extent = (-246400, 159990,
               -266400, 140000)
ax.imshow(img, extent=axis_extent)

for player_key in games[0].players:
    player = games[0].players[player_key]
    player_indices = player.tick_indices
    pos_x = games[0].id_data['pos_x'][player_indices]
    pos_y = games[0].id_data['pos_y'][player_indices]
    ax.scatter(pos_x, pos_y, c=np.random.rand(3,), s=1)

In [ ]:
class Action():
    def __init__(self, timestamp):
        self.timestamp = timestamp

In [ ]:
# Calculate next probable kill event
# For every 1000 ticks, calculate player distances to each other and their rotations
game = games[0]

def calculateDistances(team1_positions, team2_positions):
    # positions tuples of (x,y) positions
    distances = np.zeros((len(team1_positions), len(team2_positions)))
    for i in range(0, len(team1_positions)):
        for i in range(0, len(team2_positions)):
            if i == j:
                
            pos1 = np.array(team1_positions(i))
            pos2 = np.array(team2_positions(j))
            distance = np.linalg.norm(pos1, pos2)
            distances[i,j] = distance
            
    return distances

def calculateRotations(team1_rotations, team2_rotations):
    facing = np.zeros((len(team1_rotations), len(team2_rotations)))
    for i in range(0, len(team1_rotations)):
        for i in range(0, len(team2_rotations)):
            rot1 = np.array(team1_rotations(i))
            rot2 = np.array(team2_rotations(j))
            
            indicator = np.dot(rot1, rot2)
            if indicator < -0.95 and indicator > -1.05 and np.linalg.norm(rot1 + rot2) < 1:
                facing[i,j] = 1
    
    return facing



data = np.multiply(facing, distances)
engagements = np.where(data < 6000)

print engagements

In [ ]:
kill_distances = []
for game in games:
    for event in game.events:
        event = game.events[event]
        killer_pos = event.player_kill_pos(game.id_data)
        victim_pos = event.player_death_pos(game.id_data)
        dist = np.linalg.norm(np.array(killer_pos) - np.array(victim_pos))
        kill_distances.append(dist)
        
np.average(kill_distances)

In [ ]:
np.average(kill_distances)